In [15]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Layer, Average
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [16]:
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
class FuzzyLayer(Layer):
    def __init__(self, units=1, **kwargs):
        super(FuzzyLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.centers = self.add_weight(shape=(self.units, input_shape[-1]), 
                                     initializer='uniform', 
                                     trainable=True)
        self.sigmas = self.add_weight(shape=(self.units, input_shape[-1]), 
                                    initializer='ones', 
                                    trainable=True)

    def call(self, inputs):
        inputs_exp = tf.expand_dims(inputs, 1)
        centers_exp = tf.expand_dims(self.centers, 0)
        sigmas_exp = tf.expand_dims(self.sigmas, 0)
        fuzzy_output = tf.exp(-tf.square(inputs_exp - centers_exp) / (2 * tf.square(sigmas_exp)))
        return tf.reduce_sum(fuzzy_output, axis=-1)

    def get_config(self):
        config = super(FuzzyLayer, self).get_config()
        config.update({'units': self.units})
        return config

In [17]:
# To load the models later
loaded_mv2 = tf.keras.models.load_model('fuzzy_model_mv2.keras', 
                                      custom_objects={'FuzzyLayer': FuzzyLayer})
loaded_resnet = tf.keras.models.load_model('fuzzy_model_resnet.keras', 
                                        custom_objects={'FuzzyLayer': FuzzyLayer})
loaded_effnet = tf.keras.models.load_model('fuzzy_model_effnet.keras', 
                                        custom_objects={'FuzzyLayer': FuzzyLayer})

In [26]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2



from tensorflow.keras.preprocessing import image
import joblib

# Define the path to your enhanced images
ENHANCED_IMAGE_PATH = r'C:\Users\mitta\OneDrive\Desktop\pbl enhanced\enhanced_image'
def get_img_array(img_path, size):
    """Load and preprocess image"""
    img = image.load_img(img_path, target_size=size)
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array / 255.

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate Grad-CAM heatmap"""
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_output = predictions[:, pred_index]

    grads = tape.gradient(class_output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, model, last_conv_layer_name, img_size=(224, 224)):
    """Display Grad-CAM only if pneumonia is predicted"""
    img_array = get_img_array(img_path, img_size)
    
    # Make prediction first
    mv2 = loaded_mv2.predict(img_array)[0][0]
    resnet = loaded_resnet.predict(img_array)[0][0]
    effnet = loaded_effnet.predict(img_array)[0][0]
    
    pred = (3 * mv2 + 0.5 * effnet + 1 * resnet) / (3 + 0.5 + 1)
    confidence = pred if pred > 0.5 else (1 - pred)
    print(pred)
    print(mv2)
    print(resnet)
    print(effnet)
    
    # Only proceed if pneumonia is predicted with confidence > 50%
    if pred > 0.5:
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
        heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

        superimposed_img = cv2.addWeighted(img, 0.6, heatmap_colored, 0.4, 0)

        # Show the image with label
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
        plt.title(f"Pneumonia Detected ({pred*100:.2f}% confidence)", fontsize=16)
        plt.axis('off')
        plt.show()
    else:
        print(f"Normal case detected ({(1-pred)*100:.2f}% confidence). No visualization shown.")

# Example usage:
# Load your trained model (replace with your actual model loading code)
# model = tf.keras.models.load_model('your_model_path.h5')

# Define the last convolutional layer name for each model
last_conv_layers = {
    'MobileNetV2': 'out_relu',  # Typical last conv layer for MobileNetV2
    'ResNet50': 'conv5_block3_out',  # For ResNet50
    'EfficientNetB0': 'top_activation'  # For EfficientNetB0
}

# Choose which model to visualize (change as needed)
model_name = 'MobileNetV2'  # or 'ResNet50' or 'EfficientNetB0'
last_conv_layer_name_mv2 = last_conv_layers[model_name]
last_conv_layer_name_resnet = last_conv_layers['ResNet50']
last_conv_layer_name_effnet = last_conv_layers['EfficientNetB0']


# Path to a test image (replace with your actual image path)
test_image_path = input("Enter Image Path")
# Run visualization (only shows output if pneumonia is predicted)
display_gradcam(test_image_path, loaded_mv2, last_conv_layer_name_mv2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
0.4342688222726186
0.28937775
0.84567064
0.48081163
Normal case detected (56.57% confidence). No visualization shown.
